In [1]:
import collections
import requests
import json
import re
import os
import sys

In [2]:
with open('../../bundestagsreden parser/speeches_20.jsonl', 'r', encoding='utf8') as fp:
    data = list(fp)
speeches = []
for line in data:
    speeches.append(json.loads(line))

In [3]:
def filter_for(what, search_terms, speeches):
    filtered_speeches = []
    if what == 'text':
        search_terms_low = []
        for st in search_terms:
            search_terms_low.append(st.lower())
        for speech in speeches:
            match = [st in speech[what].lower() for st in search_terms_low]
            #if all(st == True for st in match):
            if any(st in speech[what] for st in search_terms):
                #print(match)
            #if ( search_terms in speech[what] ):
                filtered_speeches.append(speech)
    else:
        for speech in speeches:
            if (speech[what] in set(search_terms) ):
                filtered_speeches.append(speech)
        
    filtered_speeches.sort(key = lambda x:x['date'])   
    return filtered_speeches

In [4]:
# globale items
regex = "{(.*?)}" # hier die genaue Definition der Kommentar in vom Parser <<< >>>
kommentare = []

In [9]:
def extract_comments(speeches):
    comments =[]
    modified_speeches = []
    for speech in speeches:
        text = speech['text']
        matches = re.findall(regex,text)
        speech['comments'] = matches
        modified_speeches.append(speech)
    return modified_speeches
speeches_comments = extract_comments(speeches)
print(speeches_comments[0]['comments'])
print(speeches_comments[0]['party'])

['(Beifall bei der SPD sowie bei Abgeordneten des BÜNDNISSES 90/DIE GRÜNEN und der FDP)', '(Beifall bei der CDU/CSU)']
SPD


In [19]:
def filter_in_comments(search_term,speeches):
    filtered_speeches = []
    for speech in speeches:
        for comment in speech['comments']:
            if search_term in comment:
                filtered_speeches.append(speech)
                break  
    return filtered_speeches
speeches_with_comment_searchword = filter_in_comments('Beifall',speeches_comments)
print(speeches_with_comment_searchword[0])
print(len(speeches_with_comment_searchword))

{'id': 'ID20100100', 'period': '20', 'date': '2021-10-26', 'name': 'Gabriele Katzmarek', 'party': 'SPD', 'redner_id': '11004325', 'text': 'Sehr geehrter Herr Alterspräsident! So muss ich es jetzt sagen, genau; ich muss mich daran gewöhnen. Nicht lange. Nicht lange; da haben Sie schon recht. – Liebe Kolleginnen und Kollegen! Meine sehr geehrten Damen und Herren! Die Übernahme der Geschäftsordnung ist eine der ersten wichtigen Entscheidungen, die wir heute zu Beginn der Wahlperiode treffen müssen. Die Geschäftsordnung ist Grundlage für unsere gemeinsame Arbeit. Sie hat sich über viele Wahlperioden bewährt und wird uns auch durch die neue Wahlperiode tragen. Es ist gute Tradition, dass wir die Geschäftsordnung am Anfang einer Wahlperiode mit breiter parlamentarischer Mehrheit übernehmen, und das sollten wir auch heute tun. Sie ist nicht in Stein gemeißelt – das wissen diejenigen, die bereits in der letzten Wahlperiode hier im Bundestag tätig waren –, sondern die Geschäftsordnung ist immer

In [26]:
def filter_multiple_terms_in_comments(search_terms,speeches):
    filtered_speeches = []
    for speech in speeches:
        for comment in speech['comments']:
            if all([term in comment for term in search_terms]):
                filtered_speeches.append(speech)
                break  
    return filtered_speeches
speeches_with_conjunction_search = filter_multiple_terms_in_comments(['Beifall','AfD','lachen'],speeches_comments)
print(speeches_with_conjunction_search[0])
print(len(speeches_with_conjunction_search))

{'id': 'ID20901700', 'period': '20', 'date': '2021-12-16', 'name': 'Jürgen Braun', 'party': 'AfD', 'redner_id': '11004680', 'text': 'Frau Präsidentin! Liebe Kollegen! Unmittelbar vor Weihnachten sendet die grün-linke Ampel ein verheerendes Signal in die Welt. Mehr als 300 Millionen verfolgten Christen wird gesagt: Eure Probleme sind kein Thema für die Bundesregierung. Ich habe dem Beauftragten für Religionsfreiheit, Markus Grübel, hier vor einem halben Jahr meinen Respekt und Dank für die geleistete Arbeit ausgesprochen. Aber ich sagte schon damals, dass die Religionsfreiheit in der deutschen Politik keine Lobby hat, außer der AfD. {(Beifall bei der AfD – Lachen bei der SPD und der CDU/CSU – Michael Brand [Fulda] [CDU/CSU]: Da muss er selber lachen! Der Antrag kommt von der CDU/CSU!)} Und jetzt kann es jeder sehen: Die AfD hat wieder einmal recht behalten. Die CDU kann nicht darüber hinwegtäuschen, dass sie 16 Jahre lang Christenverfolgung kleingeredet hat. Sie hat den Begriff „Christe

In [ ]:
# Anzahl aller Kommentare
for speech in speeches:
    text = speech['text']
    matches = re.findall(regex,text)
    kommentare.extend(matches)     
len(kommentare)

In [ ]:
zähler = collections.Counter(kommentare).most_common()
zähler

In [ ]:
# Anzahl aller Kommentare
kommentare = []
for speech in speeches:
    kommentare.extend ( re.findall ( regex , speech['text'] ) ) 
print('total Kommentare      =', len(kommentare))

# Anzahl aller Beifall
word = []
for speech in speeches:
    word.extend ( re.findall ( "Beifall" , speech['text'] ) )
print('total Beifall         =', len(word))

# Anzahl Beifall CDU/CSU
word = []
for speech in speeches:
    word.extend ( re.findall ( "\((Beifall | CDU/CSU)\)" , speech['text'] ) )
print('total Beifall CDU/CSU =', len(word))

# Anzahl Beifall AfD
word = []
for speech in speeches:
    word.extend ( re.findall ( "\((Beifall | AfD)\)"  , speech['text'] ) )
print('total Beifall AfD     =', len(word))

# Anzahl aller Zurufe
word = []
for speech in speeches:
    word.extend ( re.findall ( "Zuruf" , speech['text'] ) )
print('total Zuruf           =', len(word))

# Anzahl aller Beifall der CDU
print(len(re.findall(r'\bBeifall\b|\bCDU/CSU\b',speech['text'])))

# Anzahl aller Beifall der CDU
print(len(re.findall(r'\bBeifall\b|\bAfD\b',speech['text'])))

In [ ]:
kommentare_liste = [] # Liste von Kommentare mit bestimmten Items: Redner/in, Partei, Kommentainhalt
for speech in speeches:
    text = speech['text']
    matches = re.findall(regex,text)
    for match in matches:
        kommentare_dict = {
            'name' : speech['name'],
           #'redner_id' : speech['redner_id'],
            'rednerpartei' : speech['party'],
            'inhalt' : match }
        kommentare_liste.append(kommentare_dict)
len(kommentare_liste)

In [ ]:
kommentare_liste[0] # ein Kommentar mit zusätzlichen Items

In [10]:
subset_SPD = filter_for('party',['SPD'], speeches)
len(subset_SPD)


698

In [ ]:
subset_SPD[0]